In [460]:
import numpy as np
import time

In [461]:
x=np.random.rand(12).reshape(4,3)
x

array([[ 0.65320766,  0.9249651 ,  0.03637637],
       [ 0.18326438,  0.02751804,  0.77956715],
       [ 0.43105095,  0.21254905,  0.68849988],
       [ 0.28688938,  0.62282822,  0.30081932]])

In [462]:
theta=(np.random.randn(15)*0.1).reshape(5,3)
b=np.random.rand(5).reshape(5,1)
print (theta)
print (b)

[[-0.00304631  0.13006276  0.11530479]
 [-0.10342723  0.00906805 -0.05504038]
 [ 0.0667158  -0.02951304  0.20235484]
 [-0.1097535  -0.04647094  0.0751252 ]
 [-0.07606797  0.10274504 -0.10120814]]
[[ 0.60271253]
 [ 0.25904749]
 [ 0.83515589]
 [ 0.06903154]
 [ 0.81687367]]


In [463]:
#dimensions of each layer
dimensions=np.array([5,4,3,2,1])

#empty array of weights of all layers 
weights=np.zeros(len(dimensions)*25).reshape(len(dimensions),25)

#empty array of biases of all layers
bias=np.zeros(len(dimensions)*5).reshape(len(dimensions),5)

#empty array of Zs of all layers
Z=np.zeros(len(dimensions)*25).reshape(len(dimensions),25)

#empty array of as of all layers
A=np.zeros(len(dimensions)*25).reshape(len(dimensions),25)

tic=time.time()
#loop for computation of z, a, theta and bias  of all layers expect input
for i in range (0,len(dimensions)):
    # array of z of each layer
    z=np.zeros(dimensions[i]*len(x)).reshape(dimensions[i],len(x))
    
    #print (z.shape)
    #print (x.shape)
    #print (theta.shape)
    
    #store all layer thetas in one array
    k=0
    for m in range(0,len(theta)):
        for n in range(0,len(theta[0])):
            weights[i][k]=theta[m][n]
            k+=1
    
    #store all layer biases in one array        
    c=0
    for f in range(0,len(b)):
        bias[i][c]=b[f][0]
        c+=1
    
    #compute z of each layer
    for j in range(0,len(theta)):
        z[j]=np.dot(x,theta[j].T)+b[j]
    z=z.T
    
    #store all layers Zs in one array
    temp=0
    for m in range(0,len(z)):
        for n in range(0,len(z[0])):
            Z[i][temp]=z[m][n]
            temp+=1
    
    #compute a 
    a=1/(1+np.exp(-z))
    
    #store all layers As in one array 
    temp=0
    for m in range(0,len(a)):
        for n in range(0,len(a[0])):
            A[i][temp]=a[m][n]
            temp+=1
    
    #print (a)
    
    #make output of each layer as input of another layer
    x=a
    
    #Go outside the loop 
    if i==(len(dimensions)-1):
        break
        
    #generate new thetas and biases for each layer
    theta=(np.random.randn(dimensions[i+1]*dimensions[i])*0.1).reshape(dimensions[i+1],dimensions[i])
    b=np.random.rand(dimensions[i]).reshape(dimensions[i],1)
toc=time.time()
print ((toc-tic)*1000,'nanoseconds')

1.005411148071289 nanoseconds
